In [1]:
import healpy as hp
from healpy.newvisufunc import projview, newprojplot
import numpy as np
import matplotlib.pyplot as plt
import yt
from unyt import cm, s, erg, sr

#import argparse
from tqdm import tqdm
import pickle

from astropy import units as u, constants  as cc

pc   = cc.pc.cgs.value
kB   = cc.k_B.cgs.value
Msun = cc.M_sun.cgs.value
G    = cc.G.cgs.value
Myr  = u.Myr.in_units("s")
eV = 1.60218e-12   # eV in erg

        Use to() instead. [warnings]


In [2]:
file_list = ["data/SILCC_hdf5_plt_cnt_1080", "data/SILCC_hdf5_plt_cnt_1140"]
#file_list = ["data/SILCC_hdf5_plt_cnt_1080"]
names = ["10.8Myr", "11.4Myr"]
nside = 64
radius = 80
odir = "."
odir_data = odir

radius_str = str(int(radius)).zfill(4)
file_suffix = f"r{radius_str}_RC"
print(file_suffix)

# numerical parameters for healpix
cx = -80.
cy = -150.
cz = 0.0
c = ([cx, cy, cz], "pc")          # new yt version
c = [cx*pc, cy*pc, cz*pc] # old yt version
print("center of the bubble at (pc) :", cx, cy, cz)

# Emissivity functions
def _j_halpha_R(field, data):
    T = data[("gas", "temperature")].v
    T4 = T / 1e4
    number_density = data[("gas", "number_density")].v
    ion_fraction = data[("flash", "ihp ")].v
    ne_np = (number_density * ion_fraction) ** 2
    return 2.82e-26 * (T4 ** (-0.942 - 0.03 * np.log(T4))) * ne_np * erg / cm**3 / s

def _j_halpha_C(field, data):
    T = data[("gas", "temperature")].v
    number_density = data[("gas", "number_density")].v
    ion_fraction = data[("flash", "ihp ")].v
    Ha_fraction = data[("flash", "iha ")]
    ne = number_density * ion_fraction
    nH = number_density * Ha_fraction

    def lambda_13(T):
        if 4000 <= T <= 25000:
            return 0.35 - 2.62e-7*T - 8.15e-11*T**2 + 6.19e-15*T**3
        elif 25000 < T <= 500000:
            return 0.276 + 4.99e-6*T - 8.85e-12*T**2 + 7.18e-18*T**3
        elif T < 4000:
            T = 4000
            return 0.35 - 2.62e-7*T - 8.15e-11*T**2 + 6.19e-15*T**3
        elif T > 500000:
            T = 500000
            return 0.276 + 4.99e-6*T - 8.85e-12*T**2 + 7.18e-18*T**3

    lambda_13_vec = np.vectorize(lambda_13)
    return (1.30e-17 / (4 * np.pi)) * (lambda_13_vec(T) / np.sqrt(T)) * np.exp(-12.1 * eV / kB /  T) * ne * nH * erg / cm**3 / s

def _j_halpha_RC(field, data):
    j_halpha_R = data[("gas", "j_halpha_R")]
    j_halpha_C = data[("gas", "j_halpha_C")]
    return (j_halpha_R + j_halpha_C)

def _j_halpha_RC_vol(field, data):
    j_halpha_RC = data[("gas", "j_halpha_RC")]
    cell_volume = data[("gas", "cell_volume")]
    return j_halpha_RC * cell_volume

def _j_halpha_R_vol(field, data):
    j_halpha_R = data[("gas", "j_halpha_R")]
    cell_volume = data[("gas", "cell_volume")]
    return j_halpha_R * cell_volume

def _j_halpha_C_vol(field, data):
    j_halpha_C = data[("gas", "j_halpha_C")]
    cell_volume = data[("gas", "cell_volume")]
    return j_halpha_C * cell_volume

# Dictionary to store processed data
all_data = {}

# Loop over all files
for files in file_list:
    print("processing file", files)
    ds = yt.load(files)
    yt.add_field(("gas", "j_halpha_R"), function=_j_halpha_R, units="erg/(cm**3*s)", sampling_type="cell")
    yt.add_field(("gas", "j_halpha_C"), function=_j_halpha_C, units="erg/(cm**3*s)", sampling_type="cell")
    yt.add_field(("gas", "j_halpha_RC"), function=_j_halpha_RC, units="erg/(cm**3*s)", sampling_type="cell")
    yt.add_field(("gas", "j_halpha_RC_vol"), function=_j_halpha_RC_vol, units="erg/s", sampling_type="cell")
    yt.add_field(("gas", "j_halpha_R_vol"), function=_j_halpha_R_vol, units="erg/s", sampling_type="cell")
    yt.add_field(("gas", "j_halpha_C_vol"), function=_j_halpha_C_vol, units="erg/s", sampling_type="cell")

    # select the region around the bubble
    sp = ds.sphere(c, (radius, "pc"))

    print()
    print("min and max positions of the selected region")
    print("min/max x (pc): ", np.min(sp[("gas", "x")].in_units("pc").v), np.max(sp[("gas", "x")].in_units("pc").v))
    print("min/max y (pc): ", np.min(sp[("gas", "y")].in_units("pc").v), np.max(sp[("gas", "y")].in_units("pc").v))
    print("min/max z (pc): ", np.min(sp[("gas", "z")].in_units("pc").v), np.max(sp[("gas", "z")].in_units("pc").v))

    # box size in pc
    Lx = ds.domain_width[0].in_units("pc").v
    Ly = ds.domain_width[1].in_units("pc").v
    Lz = ds.domain_width[2].in_units("pc").v

    # find coordinates with respect to the center of the sphere taking into account periodic boundaries in x and y
    posx_ctr = (sp[("gas", "x")].in_units("pc").v - cx + Lx/2.) % Lx - Lx/2.
    posy_ctr = (sp[("gas", "y")].in_units("pc").v - cy + Ly/2.) % Ly - Ly/2.
    posz_ctr =  sp[("gas", "z")].in_units("pc").v - cz
    print()
    print("min and max positions of the selected region - shifted to center of the sphere")
    print("min/max x (pc): ", np.min(posx_ctr), np.max(posx_ctr))
    print("min/max y (pc): ", np.min(posy_ctr), np.max(posy_ctr))
    print("min/max z (pc): ", np.min(posz_ctr), np.max(posz_ctr))
    # radius based on centre of mass
    rad_ctr = np.sqrt(posx_ctr**2 + posy_ctr**2 + posz_ctr**2)
    # print min max of radius
    print()
    print("min/max r (pc): ", np.min(rad_ctr), np.max(rad_ctr))

    # cell volume
    volu = sp[("gas", "cell_volume")].in_units("pc**3").v

    # create normal vectors for the healpix map
    vec_norm_x = posx_ctr / rad_ctr
    vec_norm_y = posy_ctr / rad_ctr
    vec_norm_z = posz_ctr / rad_ctr

    # compute angular size of all cells at the respective distance to the observer
    R = (3 * volu / (4 * np.pi))**(1./3.)
    dx = np.power(volu, 1./3.)

    # compute the angle of the cell with respect to the observer
    angle = np.arctan2(R, rad_ctr)

    # number of cells
    Ncell = rad_ctr.size
    print()
    # create three healpix maps
    NPIX = hp.nside2npix(nside)
    print("create healpix maps for various quantities with nside", nside, "and NPIX", NPIX)
    coldens_map     = np.zeros(NPIX) # column density
    coldens_ihp_map = np.zeros(NPIX) # column density (ionized gas)
    coldens_iha_map = np.zeros(NPIX) # column density (neutral gas)
    coldens_ih2_map = np.zeros(NPIX) # column density (molecular gas)
    Halpha_RC_lum_map  = np.zeros(NPIX) # Halpha emission luminosity
    Halpha_RC_flux_map = np.zeros(NPIX) # Halpha emission flux
    Halpha_R_flux_map = np.zeros(NPIX)
    Halpha_C_flux_map = np.zeros(NPIX)
    frac_ihp_map = np.zeros(NPIX)
    frac_iha_map = np.zeros(NPIX)
    frac_ih2_map = np.zeros(NPIX)

    total_mass = 0.0
    total_volu = 0.0
    total_halpha_RC = 0.0
    total_halpha_R = 0.0
    total_halpha_C = 0.0

    print()
    for i in tqdm(range(Ncell)):

        # find the pixels in the healpix map
        pix = hp.query_disc(nside, [vec_norm_x[i], vec_norm_y[i], vec_norm_z[i]], angle[i], inclusive=True)

        # total column density
        coldens_map[pix]   += sp[("gas", "cell_mass")][i].v / (np.pi*(R[i]*pc)**2) / len(pix)
        # column density of ionized gas
        coldens_ihp_map[pix] += sp[("gas", "cell_mass")][i].v * sp[("flash", "ihp ")][i].v / (np.pi*(R[i]*pc)**2) / len(pix)
        # column density of atomic h
        coldens_iha_map[pix] += sp[("gas", "cell_mass")][i].v * sp[("flash", "iha ")][i].v / (np.pi*(R[i]*pc)**2) / len(pix)
        # column density of h2
        coldens_ih2_map[pix] += sp[("gas", "cell_mass")][i].v * sp[("flash", "ih2 ")][i].v / (np.pi*(R[i]*pc)**2) / len(pix)
        # Halpha emission luminosity RC
        Halpha_RC_lum_map[pix] += sp[("gas", "j_halpha_RC_vol")][i].v / len(pix)
        # Halpha flux RC
        Halpha_RC_flux_map[pix] += sp[("gas", "j_halpha_RC_vol")][i].v / (4 * np.pi*(R[i]*pc)**2)
        # Halpha flux R
        Halpha_R_flux_map[pix] += sp[("gas", "j_halpha_R_vol")][i].v / (4 * np.pi*(R[i]*pc)**2)
        # Halpha flux C
        Halpha_C_flux_map[pix] += sp[("gas", "j_halpha_C_vol")][i].v / (4 * np.pi*(R[i]*pc)**2)

        total_halpha_RC += sp[("gas", "j_halpha_RC_vol")][i].v
        total_halpha_R += sp[("gas", "j_halpha_R_vol")][i].v
        total_halpha_C += sp[("gas", "j_halpha_C_vol")][i].v
        total_mass += sp[("gas", "cell_mass")][i].v
        total_volu += volu[i]

    print()

    sum_halpha_RC = np.sum(Halpha_RC_lum_map)
    print("sum_halpha_RC:", sum_halpha_RC)

    #frac ihp
    frac_ihp_map = coldens_ihp_map / coldens_map
    #frac iha
    frac_iha_map = coldens_iha_map / coldens_map
    #frac ih2
    frac_ih2_map = coldens_ih2_map / coldens_map

    print()
    print("done with the loop over cells")

    # Prepare data dictionary for this file
    data = {}

    data["simtime"] = ds.current_time.v
    data["simtime_Myr"] = ds.current_time.in_units("Myr").v
    data["Column density"] = coldens_map
    data["Column density ihp"] = coldens_ihp_map
    data["Column density iha"] = coldens_iha_map
    data["Column density ih2"] = coldens_ih2_map
    data["Halpha_RC luminosity"] = Halpha_RC_lum_map
    data["Halpha_RC flux"] = Halpha_RC_flux_map
    data["Halpha_R flux"] = Halpha_R_flux_map
    data["Halpha_C flux"] = Halpha_C_flux_map
    data["Fraction ihp"] = frac_ihp_map
    data["Fraction iha"] = frac_iha_map
    data["Fraction ih2"] = frac_ih2_map
    data["total_mass"] = total_mass
    data["total_volu"] = total_volu
    data["total_Halpha_RC"] = total_halpha_RC
    data["total_Halpha_R"] = total_halpha_R
    data["total_Halpha_C"] = total_halpha_C
    data["file_suffix"] = file_suffix  # Store file suffix
    data["filename"] = files
    # Store the data for this file
    all_data[files] = data

    # write data to disk using pickle
    with open(f"{odir_data}/{files}-{file_suffix}-data.pkl", "wb") as f:
        pickle.dump(data, f)
# Save the all_data dictionary to a file
with open("all_data.pkl", "wb") as f:
    pickle.dump(all_data, f)

print("Data processing complete.  Run the next cell for plotting.")


r0080_RC
center of the bubble at (pc) : -80.0 -150.0 0.0
processing file data/SILCC_hdf5_plt_cnt_1080


yt : [INFO     ] 2025-03-14 13:19:42,659 Parameters: current_time              = 340597785179995.1
yt : [INFO     ] 2025-03-14 13:19:42,660 Parameters: domain_dimensions         = [8 8 8]
yt : [INFO     ] 2025-03-14 13:19:42,662 Parameters: domain_left_edge          = [-7.715e+20 -7.715e+20 -7.715e+20]
yt : [INFO     ] 2025-03-14 13:19:42,663 Parameters: domain_right_edge         = [7.715e+20 7.715e+20 7.715e+20]
yt : [INFO     ] 2025-03-14 13:19:42,664 Parameters: cosmological_simulation   = 0



min and max positions of the selected region
min/max x (pc):  -159.68464626739294 -0.4883322515822414
min/max y (pc):  -229.02782599207123 -70.808176479425
min/max z (pc):  -77.15649574999414 79.1098247563231

min and max positions of the selected region - shifted to center of the sphere
min/max x (pc):  -79.68464626739294 79.51166774841775
min/max y (pc):  -79.02782599207123 79.19182352057501
min/max z (pc):  -77.15649574999414 79.1098247563231

min/max r (pc):  1.4392925289249479 79.99999710455978

create healpix maps for various quantities with nside 64 and NPIX 49152



100%|██████████| 1293308/1293308 [04:52<00:00, 4414.21it/s]
yt : [INFO     ] 2025-03-14 13:24:50,949 Parameters: current_time              = 359537415765615.6
yt : [INFO     ] 2025-03-14 13:24:50,951 Parameters: domain_dimensions         = [8 8 8]
yt : [INFO     ] 2025-03-14 13:24:50,954 Parameters: domain_left_edge          = [-7.715e+20 -7.715e+20 -7.715e+20]
yt : [INFO     ] 2025-03-14 13:24:50,959 Parameters: domain_right_edge         = [7.715e+20 7.715e+20 7.715e+20]
yt : [INFO     ] 2025-03-14 13:24:50,961 Parameters: cosmological_simulation   = 0
yt : [WARNING  ] 2025-03-14 13:24:50,964 Field ('gas', 'j_halpha_R') already exists. To override use `force_override=True`.
yt : [WARNING  ] 2025-03-14 13:24:50,965 Field ('gas', 'j_halpha_C') already exists. To override use `force_override=True`.
yt : [WARNING  ] 2025-03-14 13:24:50,966 Field ('gas', 'j_halpha_RC') already exists. To override use `force_override=True`.
yt : [WARNING  ] 2025-03-14 13:24:50,968 Field ('gas', 'j_halpha_RC


sum_halpha_RC: 2.5172438475462023e+34

done with the loop over cells
processing file data/SILCC_hdf5_plt_cnt_1140

min and max positions of the selected region
min/max x (pc):  -159.68464626739294 -0.4883322515822414
min/max y (pc):  -229.02782599207123 -70.808176479425
min/max z (pc):  -77.15649574999414 79.1098247563231

min and max positions of the selected region - shifted to center of the sphere
min/max x (pc):  -79.68464626739294 79.51166774841775
min/max y (pc):  -79.02782599207123 79.19182352057501
min/max z (pc):  -77.15649574999414 79.1098247563231

min/max r (pc):  1.4392925289249479 79.99999710455978

create healpix maps for various quantities with nside 64 and NPIX 49152



100%|██████████| 1182896/1182896 [04:28<00:00, 4404.63it/s]



sum_halpha_RC: 4.0918597901426845e+33

done with the loop over cells
Data processing complete.  Run the next cell for plotting.


In [5]:
# Load the data
with open("all_data.pkl", "rb") as f:
    all_data = pickle.load(f)

odir = "."  # Output directory
names = ["10.8Myr", "11.4Myr"] #list of names
file_list = ["data/SILCC_hdf5_plt_cnt_1080", "data/SILCC_hdf5_plt_cnt_1140"]
# Loop through the processed data and create plots
for files, data in all_data.items():

    file_suffix = data["file_suffix"] #load file suffix
    coldens_map = data["Column density"]
    coldens_ihp_map = data["Column density ihp"]
    coldens_iha_map = data["Column density iha"]
    coldens_ih2_map = data["Column density ih2"]
    Halpha_RC_lum_map = data["Halpha_RC luminosity"]
    Halpha_RC_flux_map = data["Halpha_RC flux"]
    Halpha_R_flux_map = data["Halpha_R flux"]
    Halpha_C_flux_map = data["Halpha_C flux"]
    frac_ihp_map = data["Fraction ihp"]
    frac_iha_map = data["Fraction iha"]
    frac_ih2_map = data["Fraction ih2"]
    
    # Loop over the fields to plot
    for name, field, norm, unit, vmin, vmax in zip(["Column density", "Column density ihp", "Column density iha", "Column density ih2", "Halpha_RC luminosity", "Halpha_RC flux", "Halpha_R flux", "Halpha_C flux", "Fraction ihp", "Fraction iha", "Fraction ih2"],
    [coldens_map, coldens_ihp_map, coldens_iha_map, coldens_ih2_map, Halpha_RC_lum_map, Halpha_RC_flux_map, Halpha_R_flux_map, Halpha_C_flux_map, frac_ihp_map, frac_iha_map, frac_ih2_map],
    [True, True, True, True, True, True, True, True, True, True, True],
    ["g/cm**2", "g/cm**2", "g/cm**2", "g/cm**2", "erg/s", "erg/s/cm**2", "erg/s/cm**2", "erg/s/cm**2", "", "", ""],
    [-8, -8, -11, -18, 24, -12, -12, -12, -4, -8, -10],
    [-2, -4, -2, -2, 32, -6, -6, -6,  0, 0, -0]):
                                                   
  #  [-8, -8, -11, -18, 24.6, -13.6, -13.8, -18.2, -3.8, -7.7, -10],
   # [-2.1, -3.9, -2.3, -2.7, 31.7, -5.9, -6, -6.4,  0, 0, -0.55]):
        
        # create a mollweide projection for all fields
        fig = plt.figure(figsize=(10, 5))
        if norm:
            plt_field = np.log10(field).copy()
            unit_label = f"log10({unit})" if unit else "log10"
        else:
            plt_field = field.copy()

        #hp.mollview(plt_field, return_projected_map=True, title=name, cbar=False)

        #hp.mollview(plt_field, return_projected_map=True, title=f"{name} {names[file_list.index(files)]}", cbar=False, vmin=  , vmax= )
        #hp.graticule()
        projview(
            plt_field,
            coord=["G"],
            graticule=True,
            graticule_labels=True,
            unit=f"{name} [{unit_label}]",
            xlabel="Longitude",
            ylabel="Latitude",
            cb_orientation="vertical",
            min=vmin,
            max=vmax,
            #min=np.min(plt_field),
            #max=np.max(plt_field),
            latitude_grid_spacing=45,
            projection_type="mollweide",
            title=f"{name} {names[file_list.index(files)]}",
            flip="geo",
            phi_convention="clockwise")

        #ax = plt.gca()
        #image = ax.get_images()[0]
        #cbar = fig.colorbar(image, ax=ax, orientation='vertical', shrink=0.8)
        #cbar.set_label(f"{name} [{unit_label}]", rotation=270, labelpad=20)
        #cbar.set_label(name, rotation=270, labelpad=20)
        plt.savefig(f'{odir}/{files}-{file_suffix}-{name}.pdf', bbox_inches="tight")
        plt.close()

print("Plotting complete.")


/home/neutrino/anaconda3/envs/yt/lib/python3.13/site-packages/healpy/newvisufunc.py:448: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(
/tmp/ipykernel_307039/1902396041.py:36: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=(10, 5))


Plotting complete.


<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

...

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

In [9]:
#um sicher zu sein

file_list = ["data/SILCC_hdf5_plt_cnt_1080", "data/SILCC_hdf5_plt_cnt_1140"]
#file_list = ["data/SILCC_hdf5_plt_cnt_1080"]
names = ["10.8Myr", "11.4Myr"]
nside = 64
radius = 80
odir = "."  
odir_data = odir  

radius_str = str(int(radius)).zfill(4)
file_suffix = f"r{radius_str}_RC"
print(file_suffix)

# numerical parameters for healpix
cx = -80.
cy = -150.
cz = 0.0
c = ([cx, cy, cz], "pc")          # new yt version
c = [cx*pc, cy*pc, cz*pc] # old yt version
print("center of the bubble at (pc) :", cx, cy, cz)

# emissivity Halpha 
def _j_halpha_R(field, data):
    T = data[("gas", "temperature")].v
    T4 = T / 1e4
    number_density = data[("gas", "number_density")].v
    ion_fraction = data[("flash", "ihp ")].v
    ne_np = (number_density * ion_fraction) ** 2
    return 2.82e-26 * (T4 ** (-0.942 - 0.03 * np.log(T4))) * ne_np * erg / cm**3 / s 

def _j_halpha_C(field, data):
    T = data[("gas", "temperature")].v
    number_density = data[("gas", "number_density")].v
    ion_fraction = data[("flash", "ihp ")].v
    Ha_fraction = data[("flash", "iha ")]
    ne = number_density * ion_fraction
    nH = number_density * Ha_fraction
        
    def lambda_13(T):
        if 4000 <= T <= 25000:
            return 0.35 - 2.62e-7*T - 8.15e-11*T**2 + 6.19e-15*T**3
        elif 25000 < T <= 500000:
            return 0.276 + 4.99e-6*T - 8.85e-12*T**2 + 7.18e-18*T**3
        elif T < 4000:
            T = 4000
            return 0.35 - 2.62e-7*T - 8.15e-11*T**2 + 6.19e-15*T**3
        elif T > 500000: 
            T = 500000
            return 0.276 + 4.99e-6*T - 8.85e-12*T**2 + 7.18e-18*T**3
    
    lambda_13_vec = np.vectorize(lambda_13)
    return (1.30e-17 / (4 * np.pi)) * (lambda_13_vec(T) / np.sqrt(T)) * np.exp(-12.1 * eV / kB /  T) * ne * nH * erg / cm**3 / s 

def _j_halpha_RC(field, data):
    j_halpha_R = data[("gas", "j_halpha_R")]
    j_halpha_C = data[("gas", "j_halpha_C")]
    return (j_halpha_R + j_halpha_C) 

def _j_halpha_RC_vol(field, data):
    j_halpha_RC = data[("gas", "j_halpha_RC")]
    cell_volume = data[("gas", "cell_volume")]
    return j_halpha_RC * cell_volume

def _j_halpha_R_vol(field, data):
    j_halpha_R = data[("gas", "j_halpha_R")]
    cell_volume = data[("gas", "cell_volume")]
    return j_halpha_R * cell_volume

def _j_halpha_C_vol(field, data):
    j_halpha_C = data[("gas", "j_halpha_C")]
    cell_volume = data[("gas", "cell_volume")]
    return j_halpha_C * cell_volume

# loop over all files
for files in file_list:
    print("processing file", files)
    ds = yt.load(files)
    yt.add_field(("gas", "j_halpha_R"), function=_j_halpha_R, units="erg/(cm**3*s)", sampling_type="cell")
    yt.add_field(("gas", "j_halpha_C"), function=_j_halpha_C, units="erg/(cm**3*s)", sampling_type="cell")
    yt.add_field(("gas", "j_halpha_RC"), function=_j_halpha_RC, units="erg/(cm**3*s)", sampling_type="cell")
    yt.add_field(("gas", "j_halpha_RC_vol"), function=_j_halpha_RC_vol, units="erg/s", sampling_type="cell")
    yt.add_field(("gas", "j_halpha_R_vol"), function=_j_halpha_R_vol, units="erg/s", sampling_type="cell")
    yt.add_field(("gas", "j_halpha_C_vol"), function=_j_halpha_C_vol, units="erg/s", sampling_type="cell")
   
    # select the region around the bubble
    sp = ds.sphere(c, (radius, "pc"))
    
    print()
    print("min and max positions of the selected region")
    print("min/max x (pc): ", np.min(sp[("gas", "x")].in_units("pc").v), np.max(sp[("gas", "x")].in_units("pc").v))
    print("min/max y (pc): ", np.min(sp[("gas", "y")].in_units("pc").v), np.max(sp[("gas", "y")].in_units("pc").v))
    print("min/max z (pc): ", np.min(sp[("gas", "z")].in_units("pc").v), np.max(sp[("gas", "z")].in_units("pc").v))

    # box size in pc
    Lx = ds.domain_width[0].in_units("pc").v
    Ly = ds.domain_width[1].in_units("pc").v
    Lz = ds.domain_width[2].in_units("pc").v

    # find coordinates with respect to the center of the sphere taking into account periodic boundaries in x and y
    posx_ctr = (sp[("gas", "x")].in_units("pc").v - cx + Lx/2.) % Lx - Lx/2.
    posy_ctr = (sp[("gas", "y")].in_units("pc").v - cy + Ly/2.) % Ly - Ly/2.
    posz_ctr =  sp[("gas", "z")].in_units("pc").v - cz
    print()
    print("min and max positions of the selected region - shifted to center of the sphere")
    print("min/max x (pc): ", np.min(posx_ctr), np.max(posx_ctr))
    print("min/max y (pc): ", np.min(posy_ctr), np.max(posy_ctr))
    print("min/max z (pc): ", np.min(posz_ctr), np.max(posz_ctr))
    # radius based on centre of mass
    rad_ctr = np.sqrt(posx_ctr**2 + posy_ctr**2 + posz_ctr**2)
    # print min max of radius
    print()
    print("min/max r (pc): ", np.min(rad_ctr), np.max(rad_ctr))

    # cell volume
    volu = sp[("gas", "cell_volume")].in_units("pc**3").v

    # create normal vectors for the healpix map
    vec_norm_x = posx_ctr / rad_ctr
    vec_norm_y = posy_ctr / rad_ctr
    vec_norm_z = posz_ctr / rad_ctr

    # compute angular size of all cells at the respective distance to the observer
    R = (3 * volu / (4 * np.pi))**(1./3.)
    dx = np.power(volu, 1./3.)

    # compute the angle of the cell with respect to the observer    
    angle = np.arctan2(R, rad_ctr)

    # number of cells
    Ncell = rad_ctr.size

    print()
    # create three healpix maps
    NPIX = hp.nside2npix(nside)
    print("create healpix maps for various quantities with nside", nside, "and NPIX", NPIX)
    coldens_map     = np.zeros(NPIX) # column density
    coldens_ihp_map = np.zeros(NPIX) # column density (ionized gas)
    coldens_iha_map = np.zeros(NPIX) # column density (neutral gas)
    coldens_ih2_map = np.zeros(NPIX) # column density (molecular gas)
    Halpha_RC_lum_map  = np.zeros(NPIX) # Halpha emission luminosity
    #Halpha_R_lum_map = np.zeros(NPIX)
    #Halpha_C_lum_map = np.zeros(NPIX)
    Halpha_RC_flux_map = np.zeros(NPIX) # Halpha emission flux
    Halpha_R_flux_map = np.zeros(NPIX) 
    Halpha_C_flux_map = np.zeros(NPIX)
    frac_ihp_map = np.zeros(NPIX) 
    frac_iha_map = np.zeros(NPIX) 
    frac_ih2_map = np.zeros(NPIX) 
    
    total_mass = 0.0
    total_volu = 0.0
    total_halpha_RC = 0.0
    total_halpha_R = 0.0
    total_halpha_C = 0.0

    print()
    for i in tqdm(range(Ncell)):

        # find the pixels in the healpix map
        pix = hp.query_disc(nside, [vec_norm_x[i], vec_norm_y[i], vec_norm_z[i]], angle[i], inclusive=True)

        # total column density
        coldens_map[pix]   += sp[("gas", "cell_mass")][i].v / (np.pi*(R[i]*pc)**2) / len(pix)
        # column density of ionized gas
        coldens_ihp_map[pix] += sp[("gas", "cell_mass")][i].v * sp[("flash", "ihp ")][i].v / (np.pi*(R[i]*pc)**2) / len(pix)
        # column density of atomic h
        coldens_iha_map[pix] += sp[("gas", "cell_mass")][i].v * sp[("flash", "iha ")][i].v / (np.pi*(R[i]*pc)**2) / len(pix)
        # column density of h2
        coldens_ih2_map[pix] += sp[("gas", "cell_mass")][i].v * sp[("flash", "ih2 ")][i].v / (np.pi*(R[i]*pc)**2) / len(pix)
        # Halpha emission luminosity RC
        Halpha_RC_lum_map[pix] += sp[("gas", "j_halpha_RC_vol")][i].v / len(pix)
        # Halpha flux RC
        Halpha_RC_flux_map[pix] += sp[("gas", "j_halpha_RC_vol")][i].v / (4 * np.pi*(R[i]*pc)**2) 
        # Halpha flux R
        Halpha_R_flux_map[pix] += sp[("gas", "j_halpha_R_vol")][i].v / (4 * np.pi*(R[i]*pc)**2) 
        # Halpha flux C
        Halpha_C_flux_map[pix] += sp[("gas", "j_halpha_C_vol")][i].v / (4 * np.pi*(R[i]*pc)**2) 
        
        total_halpha_RC += sp[("gas", "j_halpha_RC_vol")][i].v 
        total_halpha_R += sp[("gas", "j_halpha_R_vol")][i].v 
        total_halpha_C += sp[("gas", "j_halpha_C_vol")][i].v 
        total_mass += sp[("gas", "cell_mass")][i].v
        total_volu += volu[i]

    print()

    sum_halpha_RC = np.sum(Halpha_RC_lum_map)
    print("sum_halpha_RC:", sum_halpha_RC)
    
    #frac ihp
    frac_ihp_map = coldens_ihp_map / coldens_map
    #frac iha
    frac_iha_map = coldens_iha_map / coldens_map
    #frac ih2
    frac_ih2_map = coldens_ih2_map / coldens_map

    print()
    print("done with the loop over cells, make plots and write data to disk")

    # prepare the data field to write to disk
    data = {}

    data["simtime"] = ds.current_time.v
    data["simtime_Myr"] = ds.current_time.in_units("Myr").v

    for name, field, norm, unit, vmin, vmax in zip(["Column density", "Column density ihp", "Column density iha", "Column density ih2", "Halpha_RC luminosity", "Halpha_RC flux", "Halpha_R flux", "Halpha_C flux", "Fraction ihp", "Fraction iha", "Fraction ih2"],
    [coldens_map, coldens_ihp_map, coldens_iha_map, coldens_ih2_map, Halpha_RC_lum_map, Halpha_RC_flux_map, Halpha_R_flux_map, Halpha_C_flux_map, frac_ihp_map, frac_iha_map, frac_ih2_map],
    [True, True, True, True, True, True, True, True, True, True, True],
    ["g/cm**2", "g/cm**2", "g/cm**2", "g/cm**2", "erg/s", "erg/s/cm**2", "erg/s/cm**2", "erg/s/cm**2", "", "", ""],
    [-8, -8, -11, -18, 31.7, -13.6, -13.8, -18.2, -3.8, -7.7, -10],
    [-2.1, -3.9, -2.3, -2.7, 34.6, -5.9, -6, -6.4,  0, 0, -0.55]):
        
        # create a mollweide projection for all fields
        fig = plt.figure(figsize=(10, 5))
        if norm:
            plt_field = np.log10(field).copy()
            unit_label = f"log10({unit})" if unit else "log10"
        else:
            plt_field = field.copy()

        #hp.mollview(plt_field, return_projected_map=True, title=name, cbar=False)

        #hp.mollview(plt_field, return_projected_map=True, title=f"{name} {names[file_list.index(files)]}", cbar=False, vmin=  , vmax= )
        #hp.graticule()        
        projview(
            plt_field,
            coord=["G"],
            graticule=True,
            graticule_labels=True,
            unit=f"{name} [{unit_label}]",
            xlabel="Longitude",
            ylabel="Latitude",
            cb_orientation="vertical",
            min=vmin,  
            max=vmax,
            #min=np.min(plt_field),
            #max=np.max(plt_field),
            latitude_grid_spacing=45,
            projection_type="mollweide",
            title=f"{name} {names[file_list.index(files)]}",
            flip="geo",
            phi_convention="clockwise")

        #ax = plt.gca()
        #image = ax.get_images()[0]
        #cbar = fig.colorbar(image, ax=ax, orientation='vertical', shrink=0.8)
        #cbar.set_label(f"{name} [{unit_label}]", rotation=270, labelpad=20)
        #cbar.set_label(name, rotation=270, labelpad=20)
        plt.savefig(f'{odir}/{files}-{file_suffix}-{name}.pdf', bbox_inches="tight")
        plt.close()

        # dave the data in global dictionary
        data[name] = field

    # add global quantities to the data dictionary
    data["total_mass"] = total_mass
    data["total_volu"] = total_volu
    data["total_Halpha_RC"] = total_halpha_RC
    data["total_Halpha_R"] = total_halpha_R
    data["total_Halpha_C"] = total_halpha_C

    # write data to disk using pickle
    with open(f"{odir_data}/{files}-{file_suffix}-data.pkl", "wb") as f:
        pickle.dump(data, f)


r0080_RC
center of the bubble at (pc) : -80.0 -150.0 0.0
processing file data/SILCC_hdf5_plt_cnt_1080


yt : [INFO     ] 2025-03-13 10:35:55,159 Parameters: current_time              = 340597785179995.1
yt : [INFO     ] 2025-03-13 10:35:55,171 Parameters: domain_dimensions         = [8 8 8]
yt : [INFO     ] 2025-03-13 10:35:55,173 Parameters: domain_left_edge          = [-7.715e+20 -7.715e+20 -7.715e+20]
yt : [INFO     ] 2025-03-13 10:35:55,175 Parameters: domain_right_edge         = [7.715e+20 7.715e+20 7.715e+20]
yt : [INFO     ] 2025-03-13 10:35:55,177 Parameters: cosmological_simulation   = 0
yt : [WARNING  ] 2025-03-13 10:35:55,182 Field ('gas', 'j_halpha_R') already exists. To override use `force_override=True`.
yt : [WARNING  ] 2025-03-13 10:35:55,184 Field ('gas', 'j_halpha_C') already exists. To override use `force_override=True`.
yt : [WARNING  ] 2025-03-13 10:35:55,186 Field ('gas', 'j_halpha_RC') already exists. To override use `force_override=True`.
yt : [WARNING  ] 2025-03-13 10:35:55,187 Field ('gas', 'j_halpha_RC_vol') already exists. To override use `force_override=True`


min and max positions of the selected region
min/max x (pc):  -159.68464626739294 -0.4883322515822414
min/max y (pc):  -229.02782599207123 -70.808176479425
min/max z (pc):  -77.15649574999414 79.1098247563231

min and max positions of the selected region - shifted to center of the sphere
min/max x (pc):  -79.68464626739294 79.51166774841775
min/max y (pc):  -79.02782599207123 79.19182352057501
min/max z (pc):  -77.15649574999414 79.1098247563231

min/max r (pc):  1.4392925289249479 79.99999710455978

create healpix maps for various quantities with nside 64 and NPIX 49152



 29%|██▊       | 370062/1293308 [01:22<03:25, 4495.88it/s]


KeyboardInterrupt: 